In [1]:
import pandas as pd
import numpy as np 
import folium
import json
import math
from shapely.geometry import shape, Point
import requests
from pandas.io.json import json_normalize




In [ ]:
ukpostcode_df=pd.read_csv('http://geoportal1-ons.opendata.arcgis.com/datasets/687f346f5023410ba86615655ff33ca9_1.csv')
ukpostcode_df.shape


In [ ]:
ukpostcode_df.head()

In [ ]:
ukgeo = r'Local_Authority_Districts__December_2017__Boundaries_in_the_UK__WGS84_.geojson'



### Data
* Counties and unitary Athorities geo json *https://opendata.arcgis.com/datasets/a8531598f29f44e7ad455abb6bf59c60_0.geojson*
    http://geoportal1-ons.opendata.arcgis.com/datasets/fab4feab211c4899b602ecfbfbc420a3_2.geojson?outSR={%22latestWkid%22:4326,%22wkid%22:4326}
* Counties population *https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland*
* Counties area *https://www.arcgis.com/sharing/rest/content/items/e919e5a1647f426595650429937b7ad5/data*
* Density by local authority area *https://www.ons.gov.uk/visualisations/dvc809/data/datadownload.xlsx*
* Crime
* Average Income

Venue types:

* Hospitals
* General Practitioner
* Pharmacy
* Train Station *https://www.theguardian.com/news/datablog/2011/may/19/train-stations-listed-rail#data*
* Supermarket


In [ ]:
# gejoson to dataframe

geodf = pd.read_csv('Local_Authority_Districts_(April_2019)_Boundaries_UK_BFE.csv')
geodf


In [2]:
LondonBoroughsCodes = list(range(9000001,9000034))
LondonBoroughs = []
for item in LondonBoroughsCodes:
    LondonBoroughs.append('E0{}'.format(item))
 


In [ ]:
geodf_london = geodf[geodf.LAD19CD.isin(LondonBoroughs)]
print(geodf_london.shape)

geodf_london.drop(['FID', 'LAD19CD', 'BNG_E', 'BNG_N'], axis=1, inplace=True)
print(geodf_london.shape)
geodf_london.columns = ['Borough_Code', 'Name', 'Long', 'Lat', 'Area', 'Length']
geodf_london.reset_index(drop=True, inplace=True)
geodf_london.head()

* Supermarket: 52f2ab2ebcbc57f1066b8b46
* Fruit & Vegetable Store: 52f2ab2ebcbc57f1066b8b1c
* Pharmacy: 4bf58dd8d48988d10f951735
* Shopping Mall: 4bf58dd8d48988d1fd941735
* Airport Terminal: 4bf58dd8d48988d1eb931735
* Train Station: 4bf58dd8d48988d129951735
* Gym / Fitness Center: 4bf58dd8d48988d175941735
* Doctor's Office: 4bf58dd8d48988d177941735
* Hospital: 4bf58dd8d48988d196941735
* Elementary School: 4f4533804b9074f6e4fb0105
* Middle School: 4f4533814b9074f6e4fb0106
* Preschool: 52e81612bcbc57f1066b7a45


In [ ]:

# depending on your version, use: from shapely.geometry import shape, Point

# load GeoJSON file containing sectors
def find_borough(names, lats, longs, geojson):
    with open(geojson) as f:
        js = json.load(f)
    venues_borough = []
    for name, lat, long, in zip(names, lats, longs):
        # construct point based on lon/lat returned by geocoder
        point = Point(long, lat)
        # check each polygon to see if it contains the point
        for feature in js['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                venues_borough.append([[name, lat, long, feature['properties']['lad17nm']]])
    venues_borough_df= pd.DataFrame([item for venue_borough in venues_borough for item in venue_borough])
    venues_borough_df.columns = ['Name', 'Latitude', 'Longitude', 'Borough']
    return venues_borough_df

In [3]:
#create a df with all the points in the london boroughs boundaries
boroughs_boundary = []
with open('Local_Authority_Districts__December_2017__Boundaries_in_the_UK__WGS84_.geojson') as f:
    data = json.load(f)
    for element in data['features']:
        if element['properties']['lad17cd']  in LondonBoroughs:
            for coordinategroup in element['geometry']['coordinates']:
                for coordinate in coordinategroup:
                    boroughs_boundary.append([[element['properties']['lad17cd'], coordinate[0], coordinate[1]]])

boroughs_boundary_df = pd.DataFrame([item for element in boroughs_boundary for item in element])
boroughs_boundary_df.columns = ['Borough_code', 'Longitude', 'Latitude']
print(boroughs_boundary_df.shape)
min_long=boroughs_boundary_df['Longitude'].min()
max_long=boroughs_boundary_df['Longitude'].max()
min_lat=boroughs_boundary_df['Latitude'].min()
max_lat=boroughs_boundary_df['Latitude'].max()


#create a new geometry by merging all London boroughs into one area



(1272, 3)


In [4]:
#Delete features for non Local Authorities outside Lonodon Area
with open('Local_Authority_Districts__December_2017__Boundaries_in_the_UK__WGS84_.geojson', 'r') as f_in:
    data2 = json.load(f_in)
    features_filtered=[]
    for i in range(len(data2['features'])):
        #print(data2[i])
        if data2['features'][i]['properties']['lad17cd']  in LondonBoroughs:
            features_filtered.append(data2['features'][i])
            
    data2['features'] = features_filtered  
    with open('London_Boroughs_Boundaries.geojson', 'w') as f_out:
        json.dump(data2, f_out)



In [5]:

radius = 2.5 #radio en km
delta_longitude = math.sqrt(3)*radius/(111.32*math.cos(math.radians(boroughs_boundary_df['Latitude'].mean())))
delta_latitude = 3*radius/110.574

mesh_points = []

for i in range(-1, int((max_long - min_long)/delta_longitude)+2):
    for j in range(-1, int((max_lat - min_lat)/delta_latitude)+2):
        mesh_points.append([min_lat + j*delta_latitude, min_long + i*delta_longitude])
        mesh_points.append([min_lat + delta_latitude/2 +j*delta_latitude, min_long +delta_longitude/2 + i*delta_longitude])
print('Mesh for London area created')
London_lat=boroughs_boundary_df['Latitude'].mean()
London_lon=boroughs_boundary_df['Longitude'].mean()
print("The coordinates for the center of Lonodon area are: {},{}".format(London_lat,London_lon))

Mesh for London area created
The coordinates for the center of Lonodon area are: 51.50181516025634,-0.12241524956441778


In [7]:
map_london = folium.Map(location=[London_lat, London_lon], zoom_start=10, tiles='openstreetmap')
map_london.choropleth(
    geo_data='London_Boroughs_Boundaries.geojson',
    
    fill_color='orange', 
    fill_opacity=0.4, 
    line_opacity=0.6,
    legend_name='London Boroughs'
)

for point in mesh_points:
   folium.Circle(point, radius= radius*1000, color='green', fill_color='green', fill_opacity=0.3).add_to(map_london)
map_london

For each point in the mesh we are going to check if the vertex of the its hexagon will be inside any london area:

In [8]:
def check_london_area(geopoint, geojson):
    with open(geojson) as f:
        js = json.load(f)
        point = Point(geopoint[1],geopoint[0])
        # check each polygon to see if it contains the point
        result=False
        for feature in js['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                return True
                break
    return result    
mesh_points_within_london = []
for point in mesh_points:
    point_a = [point[0] + radius/110.574, point[1]]
    point_b = [point[0] + radius/(2*110.574), point[1] + math.sqrt(3)*radius/(2*111.32*math.cos(math.radians(boroughs_boundary_df['Latitude'].mean())))]
    point_c = [point[0] - radius/(2*110.574), point[1] + math.sqrt(3)*radius/(2*111.32*math.cos(math.radians(boroughs_boundary_df['Latitude'].mean())))]
    point_d = [point[0] - radius/110.574, point[1]]
    point_e = [point[0] + radius/(2*110.574), point[1] - math.sqrt(3)*radius/(2*111.32*math.cos(math.radians(boroughs_boundary_df['Latitude'].mean())))]
    point_f = [point[0] - radius/(2*110.574), point[1] - math.sqrt(3)*radius/(2*111.32*math.cos(math.radians(boroughs_boundary_df['Latitude'].mean())))]
    if check_london_area(point_a, 'London_Boroughs_Boundaries.geojson'):
       mesh_points_within_london.append(point)
    elif check_london_area(point_b, 'London_Boroughs_Boundaries.geojson'):
       mesh_points_within_london.append(point)
    elif check_london_area(point_c, 'London_Boroughs_Boundaries.geojson'):
       mesh_points_within_london.append(point)
    elif check_london_area(point_d, 'London_Boroughs_Boundaries.geojson'):
       mesh_points_within_london.append(point) 
    elif check_london_area(point_e, 'London_Boroughs_Boundaries.geojson'):
       mesh_points_within_london.append(point) 
    elif check_london_area(point_f, 'London_Boroughs_Boundaries.geojson'):
       mesh_points_within_london.append(point)       
    folium.CircleMarker(point_a, radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=0.7, parse_html=False).add_to(map_london)
    folium.CircleMarker(point_b, radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=0.7, parse_html=False).add_to(map_london)
    folium.CircleMarker(point_c, radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=0.7, parse_html=False).add_to(map_london)
    folium.CircleMarker(point_d, radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=0.7, parse_html=False).add_to(map_london)
    folium.CircleMarker(point_e, radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=0.7, parse_html=False).add_to(map_london)
    folium.CircleMarker(point_f, radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=0.7, parse_html=False).add_to(map_london)
map_london


In [9]:
map_london = folium.Map(location=[London_lat, London_lon], zoom_start=10, tiles='openstreetmap')
map_london.choropleth(
    geo_data='London_Boroughs_Boundaries.geojson',
    
    fill_color='orange', 
    fill_opacity=0.4, 
    line_opacity=0.6,
    legend_name='London Boroughs'
)

for point in mesh_points_within_london:
   folium.Circle(point, radius= radius*1000, color='green', fill_color='green', fill_opacity=0.3).add_to(map_london)
map_london

In [10]:
print(len(mesh_points))
print(len(mesh_points_within_london))

256
130


In [11]:
CLIENT_ID = '3CL45KZFHOK53DH0KIK33CC2YQWHHYSYUFFVCZNMD43OEXL1' # your Foursquare ID
CLIENT_SECRET = 'FSBQZABH0LK45QVXZ1Y0ZUQU421M5SCU5XYMX1ZSIMB54LEC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

def getVenues(mesh_points, group, category_name, category_code, radius=2000):
    
    venues_list=[]
    for mesh_point in mesh_points:
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            mesh_point[0], 
            mesh_point[1], 
            VERSION,
            category_code,
            radius, 
            LIMIT)
        
        # make the GET request
        json_results = requests.get(url).json() #["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        results = json_results['response']['venues']
        
        venues_list.append([(
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            group,
            category_name) for v in results])

    
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        if nearby_venues.shape[0]>0:
            nearby_venues.columns = ['Venue', 
                        'Latitude', 
                        'Longitude',
                        'Group', 
                        'Category']
                        
            nearby_venues.drop_duplicates(inplace=True)
        nearby_venues.to_csv('{}.csv'.format(category_name))


Your credentails:
CLIENT_ID: 3CL45KZFHOK53DH0KIK33CC2YQWHHYSYUFFVCZNMD43OEXL1
CLIENT_SECRET:FSBQZABH0LK45QVXZ1Y0ZUQU421M5SCU5XYMX1ZSIMB54LEC


In [12]:
categories_group = {
    'Health': {
        'Hospital': '4bf58dd8d48988d196941735',
        'Doctor': '4bf58dd8d48988d177941735',
        'Pharmacy': '4bf58dd8d48988d10f951735'},
    'Transport': {
        'Trains_Platforms': '4f4531504b9074f6e4fb0102',
        'Underground': '4bf58dd8d48988d1fd931735',
        'Light_rail': '4bf58dd8d48988d1fc931735'},
    'Well-Being': {
        'Park': '4bf58dd8d48988d163941735',
        'Pool': '4bf58dd8d48988d15e941735',
        'Gym': '4bf58dd8d48988d175941735'},
    'Dailyneeds': {
        'Fruit_And_Veg_Shop': '52f2ab2ebcbc57f1066b8b1c',
        'Supermarket': '52f2ab2ebcbc57f1066b8b46',
        'Shopping_Mall': '4bf58dd8d48988d1fd941735'},
    'Education':{
        'Elementary_School': '4f4533804b9074f6e4fb0105',
        'Middle_School': '4f4533814b9074f6e4fb0106',
        'Preschool': '52e81612bcbc57f1066b7a45'}

    }
categories_group_1={
    'Health': {
        'Hospital': '4bf58dd8d48988d196941735',
        'Doctor': '4bf58dd8d48988d177941735',
        'Pharmacy': '4bf58dd8d48988d10f951735'},
    'Transport': {
        'Trains_Platforms': '4f4531504b9074f6e4fb0102',
        'Underground': '4bf58dd8d48988d1fd931735',
        'Light_rail': '4bf58dd8d48988d1fc931735'}
    }
categories_group_2={
    'Well-Being': {
        'Park': '4bf58dd8d48988d163941735',
        'Pool': '4bf58dd8d48988d15e941735',
        'Gym': '4bf58dd8d48988d175941735'},
    'Dailyneeds': {
        'Fruit_And_Veg_Shop': '52f2ab2ebcbc57f1066b8b1c',
        'Supermarket': '52f2ab2ebcbc57f1066b8b46',
        'Shopping_Mall': '4bf58dd8d48988d1fd941735'},
    'Education':{
        'Elementary_School': '4f4533804b9074f6e4fb0105',
        'Middle_School': '4f4533814b9074f6e4fb0106',
        'Preschool': '52e81612bcbc57f1066b7a45'}

    }


In [ ]:
trialmesh = mesh_points_within_london[:2] 
print(trialmesh)

In [ ]:
for group_name, categories in categories_group_1.items():
    print('Obtaining venues for group {}'.format(group_name))
    for name, code  in categories.items():
        print ('Using code:{} to generate {}.csv'.format(code,name))
        print ('...')
        getVenues(mesh_points_within_london, group_name, name, code, radius*1000 )


In [13]:
for group_name, categories in categories_group_2.items():
    print('Obtaining venues for group {}'.format(group_name))
    for name, code  in categories.items():
        print ('Using code:{} to generate {}.csv'.format(code,name))
        print ('...')
        getVenues(mesh_points_within_london, group_name, name, code, radius*1000 )


Obtaining venues for group Well-Being
Using code:4bf58dd8d48988d163941735 to generate Park.csv
...
Using code:4bf58dd8d48988d15e941735 to generate Pool.csv
...
Using code:4bf58dd8d48988d175941735 to generate Gym.csv
...
Obtaining venues for group Dailyneeds
Using code:52f2ab2ebcbc57f1066b8b1c to generate Fruit_And_Veg_Shop.csv
...
Using code:52f2ab2ebcbc57f1066b8b46 to generate Supermarket.csv
...
Using code:4bf58dd8d48988d1fd941735 to generate Shopping_Mall.csv
...
Obtaining venues for group Education
Using code:4f4533804b9074f6e4fb0105 to generate Elementary_School.csv
...
Using code:4f4533814b9074f6e4fb0106 to generate Middle_School.csv
...


KeyError: 'venues'

In [20]:
trains_platforms_df = pd.read_csv('Trains_Platforms.csv')
underground_df = pd.read_csv('Underground.csv')
Elementary_School_df = pd.read_csv('Elementary_School.csv')
Supermarket_df = pd.read_csv('Supermarket.csv')



In [22]:
for lat, long in zip(Elementary_School_df['Latitude'],Elementary_School_df['Longitude']):
    folium.CircleMarker([lat,long], radius=2, color='yellow', fill=True, fill_color='yellow', fill_opacity=0.7, parse_html=False).add_to(map_london)
for lat, long in zip(Supermarket_df['Latitude'],Supermarket_df['Longitude']):
    folium.CircleMarker([lat,long], radius=2, color='orange', fill=True, fill_color='orange', fill_opacity=0.7, parse_html=False).add_to(map_london)

map_london